In [2]:
import pandas as pd
import json

# Load the CSV file with predicted data
csv_file_path = r'C:\Users\Asus Vivobook\Code\py_code\GWS-Classification\data\GLDAS_test_data_with_predictions.csv'  # Replace with your CSV file path
data = pd.read_csv(csv_file_path)

# Extract coordinates from the geo column
def extract_coordinates(geo_str):
    geo_dict = json.loads(geo_str)
    return geo_dict['coordinates']

data['coordinates'] = data['geo'].apply(extract_coordinates)
data['lon'] = data['coordinates'].apply(lambda x: x[0])
data['lat'] = data['coordinates'].apply(lambda x: x[1])

data.head()

,VH,VV,elevation,slope,GWS_true,GWS_pred,geo,coordinates,lon,lat
0,0.425445,0.436948,0.073981,0.001262,1955011,1844457.718,"{""geodesic"":false,""type"":""Point"",""coordinates""...","[115.37621223961293, -0.8751395705188045]",115.376212,-0.875140
1,0.423693,0.420001,0.047919,0.000322,1061270,1401186.780,"{""geodesic"":false,""type"":""Point"",""coordinates""...","[114.62620880890154, -1.8751441448006574]",114.626209,-1.875144
2,0.432816,0.436593,0.042455,0.000379,778860,990644.790,"{""geodesic"":false,""type"":""Point"",""coordinates""...","[111.87619622962643, -3.1251498626529752]",111.876196,-3.125150
3,0.420904,0.443026,0.046238,0.000386,841846,1040704.652,"{""geodesic"":false,""type"":""Point"",""coordinates""...","[111.87619622962643, -2.3751464319415803]",111.876196,-2.375146
4,0.403037,0.432528,0.043296,0.000238,1364789,1011566.002,"{""geodesic"":false,""type"":""Point"",""coordinates""...","[112.87620080390832, -2.1251452883711224]",112.876201,-2.125145


In [3]:
import numpy as np
from scipy.interpolate import griddata

# Define the grid
grid_lon, grid_lat = np.meshgrid(
    np.linspace(data['lon'].min(), data['lon'].max(), 100),
    np.linspace(data['lat'].min(), data['lat'].max(), 100)
)

# Interpolate the data
grid_z = griddata(
    (data['lon'], data['lat']), 
    data['GWS_pred'], 
    (grid_lon, grid_lat), 
    method='cubic'
)

In [5]:
import rasterio
from rasterio.transform import from_origin

# Define the transform
transform = from_origin(data['lon'].min(), data['lat'].max(), 
                        (data['lon'].max() - data['lon'].min()) / 100, 
                        (data['lat'].max() - data['lat'].min()) / 100)

output_tiff = 'predicted_GWS.tif'
with rasterio.open(
    output_tiff, 'w', driver='GTiff',
    height=grid_z.shape[0], width=grid_z.shape[1],
    count=1, dtype=grid_z.dtype,
    crs='+proj=latlong',
    transform=transform,
) as dst:
    dst.write(grid_z, 1)


In [ ]:
import matplotlib.pyplot as plt

def convert_tiff_to_png(tiff_file_path, output_png_path):
    with rasterio.open(tiff_file_path) as src:
        data = src.read(1)
        masked_data = np.ma.masked_equal(data, src.nodata)
        plt.imshow(masked_data, cmap='jet')
        plt.colorbar()
        plt.savefig(output_png_path, bbox_inches='tight')
        plt.close()

# Convert the new TIFF to PNG
predicted_tiff = 'predicted_GWS.tif'
predicted_png = 'predicted_GWS.png'
convert_tiff_to_png(predicted_tiff, predicted_png)
png_files['predicted_GWS'] = predicted_png
